# A file for preprocessing variant data

### TODO: consider marking presence of heterozygosity as a binary feature. Could be useful for labeling data in visualizations

### Before running this, you should use GATK's VariantsToTable tool. This notebook performs additional preprocessing on the table generated by VariantsToTable.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## Remove NANs from the VQSR-filtered SNP table. This table wil be useful in case we wish to use features computed by GATK (e.g. ReadPosRankSum) for our own algorithm.

In [2]:
filename = 'NA12878.LowSeq.illumina.bwa.sorted.dedup.20.sam.wFlag.qual.recalibrated.filtered'
in_file = 'data/vqsr_output/' + filename + '.table'

df = pd.read_csv(in_file, sep='\t')
print df.columns
print df.shape

Index([u'CHROM', u'POS', u'ID', u'REF', u'ALT', u'QUAL', u'FILTER', u'AC',
       u'AF', u'AN', u'BaseQRankSum', u'ClippingRankSum', u'DB', u'DP', u'FS',
       u'MLEAC', u'MLEAF', u'MQ', u'MQ0', u'MQRankSum', u'POSITIVE_TRAIN_SITE',
       u'QD', u'ReadPosRankSum', u'SOR', u'VQSLOD', u'culprit', u'FORMAT',
       u'sample1', u'sample1.GT', u'sample1.AD', u'sample1.DP', u'sample1.GQ',
       u'sample1.PL'],
      dtype='object')
(86843, 33)


In [3]:
df.describe()

/Users/nicochaves/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,CHROM,POS,QUAL,AC,AF,AN,BaseQRankSum,ClippingRankSum,DP,FS,...,MQ0,MQRankSum,QD,ReadPosRankSum,SOR,VQSLOD,FORMAT,sample1,sample1.DP,sample1.GQ
count,86843.0,8.684300e+04,86843.000000,86843.000000,86843.000000,86843.0,59433.000000,59433.000000,86843.000000,86843.000000,...,86843.0,59433.000000,86843.000000,59402.000000,86843.000000,84521.000000,0.0,0.0,86843.000000,86843.000000
mean,20.0,3.087816e+07,305.694981,1.317343,0.658671,2.0,-0.021814,-0.017144,16.953514,2.123864,...,0.0,-0.525123,18.550534,0.100956,1.175315,14.530439,NaN,NaN,16.730767,75.993713
std,0.0,1.856545e+07,286.177600,0.465445,0.232722,0.0,1.458781,0.884873,13.412419,3.937501,...,0.0,1.435626,10.359611,1.009716,0.771909,9.284664,NaN,NaN,13.057907,28.616600
min,20.0,6.179500e+04,10.200000,1.000000,0.500000,2.0,-6.280000,-3.575000,1.000000,0.000000,...,0.0,-8.498000,0.060000,-4.343000,0.008000,-55.760000,NaN,NaN,1.000000,0.000000
25%,20.0,1.539897e+07,139.770000,1.000000,0.500000,2.0,NaN,NaN,11.000000,0.000000,...,0.0,NaN,10.310000,NaN,0.693000,NaN,NaN,NaN,11.000000,48.000000
50%,20.0,2.947232e+07,238.770000,1.000000,0.500000,2.0,NaN,NaN,14.000000,0.000000,...,0.0,NaN,16.230000,NaN,1.002000,NaN,NaN,NaN,14.000000,99.000000
75%,20.0,4.786677e+07,411.770000,2.000000,1.000000,2.0,NaN,NaN,18.000000,2.743500,...,0.0,NaN,29.620000,NaN,1.492000,NaN,NaN,NaN,18.000000,99.000000
max,20.0,6.296289e+07,6700.770000,2.000000,1.000000,2.0,6.495000,3.366000,224.000000,71.987000,...,0.0,4.704000,40.990000,3.963000,7.886000,25.200000,NaN,NaN,218.000000,99.000000


In [4]:
df.dropna(axis='columns', how='any', inplace=True)
print df.columns
df.describe()

Index([u'CHROM', u'POS', u'ID', u'REF', u'ALT', u'QUAL', u'FILTER', u'AC',
       u'AF', u'AN', u'DP', u'FS', u'MLEAC', u'MLEAF', u'MQ', u'MQ0', u'QD',
       u'SOR', u'sample1.GT', u'sample1.AD', u'sample1.DP', u'sample1.GQ',
       u'sample1.PL'],
      dtype='object')


,CHROM,POS,QUAL,AC,AF,AN,DP,FS,MLEAC,MLEAF,MQ,MQ0,QD,SOR,sample1.DP,sample1.GQ
count,86843.0,8.684300e+04,86843.000000,86843.000000,86843.000000,86843.0,86843.000000,86843.000000,86843.000000,86843.000000,86843.000000,86843.0,86843.000000,86843.000000,86843.000000,86843.000000
mean,20.0,3.087816e+07,305.694981,1.317343,0.658671,2.0,16.953514,2.123864,1.317251,0.658625,57.872924,0.0,18.550534,1.175315,16.730767,75.993713
std,0.0,1.856545e+07,286.177600,0.465445,0.232722,0.0,13.412419,3.937501,0.465409,0.232704,5.107752,0.0,10.359611,0.771909,13.057907,28.616600
min,20.0,6.179500e+04,10.200000,1.000000,0.500000,2.0,1.000000,0.000000,1.000000,0.500000,20.830000,0.0,0.060000,0.008000,1.000000,0.000000
25%,20.0,1.539897e+07,139.770000,1.000000,0.500000,2.0,11.000000,0.000000,1.000000,0.500000,60.000000,0.0,10.310000,0.693000,11.000000,48.000000
50%,20.0,2.947232e+07,238.770000,1.000000,0.500000,2.0,14.000000,0.000000,1.000000,0.500000,60.000000,0.0,16.230000,1.002000,14.000000,99.000000
75%,20.0,4.786677e+07,411.770000,2.000000,1.000000,2.0,18.000000,2.743500,2.000000,1.000000,60.000000,0.0,29.620000,1.492000,18.000000,99.000000
max,20.0,6.296289e+07,6700.770000,2.000000,1.000000,2.0,224.000000,71.987000,2.000000,1.000000,70.000000,0.0,40.990000,7.886000,218.000000,99.000000


In [5]:
out_file = 'data/processed/' + filename + '.noNAN.table'
df.to_csv(out_file, sep='\t')

In [6]:
filters = df['FILTER']
for f in filters:
    if f != "PASS":
        print f
        break

VQSRTrancheSNP99.00to99.90


# Remove NANs from the raw SNP table

In [7]:
raw_filename = 'NA12878.LowSeq.illumina.bwa.sorted.dedup.20.sam.wFlag.qual.raw.snps'
raw_in_file = 'data/processed/' + raw_filename + '.table'

raw_df = pd.read_csv(raw_in_file, sep='\t')
print raw_df.columns
print raw_df.shape
raw_df.describe()

Index([u'CHROM', u'POS', u'ID', u'REF', u'ALT', u'QUAL', u'FILTER', u'AC',
       u'AF', u'AN', u'BaseQRankSum', u'ClippingRankSum', u'DB', u'DP', u'FS',
       u'MLEAC', u'MLEAF', u'MQ', u'MQ0', u'MQRankSum', u'QD',
       u'ReadPosRankSum', u'SOR', u'culprit', u'FORMAT', u'sample1',
       u'sample1.GT', u'sample1.AD', u'sample1.DP', u'sample1.GQ',
       u'sample1.PL'],
      dtype='object')
(86843, 31)


,CHROM,POS,QUAL,AC,AF,AN,BaseQRankSum,ClippingRankSum,DP,FS,...,MQ0,MQRankSum,QD,ReadPosRankSum,SOR,culprit,FORMAT,sample1,sample1.DP,sample1.GQ
count,86843.0,8.684300e+04,86843.000000,86843.000000,86843.000000,86843.0,59433.000000,59433.000000,86843.000000,86843.000000,...,86843.0,59433.000000,86843.000000,59402.000000,86843.000000,0.0,0.0,0.0,86843.000000,86843.000000
mean,20.0,3.087816e+07,305.694981,1.317343,0.658671,2.0,-0.021814,-0.017144,16.953514,2.123864,...,0.0,-0.525123,18.550534,0.100956,1.175315,NaN,NaN,NaN,16.730767,75.993713
std,0.0,1.856545e+07,286.177600,0.465445,0.232722,0.0,1.458781,0.884873,13.412419,3.937501,...,0.0,1.435626,10.359611,1.009716,0.771909,NaN,NaN,NaN,13.057907,28.616600
min,20.0,6.179500e+04,10.200000,1.000000,0.500000,2.0,-6.280000,-3.575000,1.000000,0.000000,...,0.0,-8.498000,0.060000,-4.343000,0.008000,NaN,NaN,NaN,1.000000,0.000000
25%,20.0,1.539897e+07,139.770000,1.000000,0.500000,2.0,NaN,NaN,11.000000,0.000000,...,0.0,NaN,10.310000,NaN,0.693000,NaN,NaN,NaN,11.000000,48.000000
50%,20.0,2.947232e+07,238.770000,1.000000,0.500000,2.0,NaN,NaN,14.000000,0.000000,...,0.0,NaN,16.230000,NaN,1.002000,NaN,NaN,NaN,14.000000,99.000000
75%,20.0,4.786677e+07,411.770000,2.000000,1.000000,2.0,NaN,NaN,18.000000,2.743500,...,0.0,NaN,29.620000,NaN,1.492000,NaN,NaN,NaN,18.000000,99.000000
max,20.0,6.296289e+07,6700.770000,2.000000,1.000000,2.0,6.495000,3.366000,224.000000,71.987000,...,0.0,4.704000,40.990000,3.963000,7.886000,NaN,NaN,NaN,218.000000,99.000000


In [8]:
raw_df.dropna(axis='columns', how='any', inplace=True)
print raw_df.columns
raw_df.describe()

Index([u'CHROM', u'POS', u'ID', u'REF', u'ALT', u'QUAL', u'FILTER', u'AC',
       u'AF', u'AN', u'DP', u'FS', u'MLEAC', u'MLEAF', u'MQ', u'MQ0', u'QD',
       u'SOR', u'sample1.GT', u'sample1.AD', u'sample1.DP', u'sample1.GQ',
       u'sample1.PL'],
      dtype='object')


,CHROM,POS,QUAL,AC,AF,AN,DP,FS,MLEAC,MLEAF,MQ,MQ0,QD,SOR,sample1.DP,sample1.GQ
count,86843.0,8.684300e+04,86843.000000,86843.000000,86843.000000,86843.0,86843.000000,86843.000000,86843.000000,86843.000000,86843.000000,86843.0,86843.000000,86843.000000,86843.000000,86843.000000
mean,20.0,3.087816e+07,305.694981,1.317343,0.658671,2.0,16.953514,2.123864,1.317251,0.658625,57.872924,0.0,18.550534,1.175315,16.730767,75.993713
std,0.0,1.856545e+07,286.177600,0.465445,0.232722,0.0,13.412419,3.937501,0.465409,0.232704,5.107752,0.0,10.359611,0.771909,13.057907,28.616600
min,20.0,6.179500e+04,10.200000,1.000000,0.500000,2.0,1.000000,0.000000,1.000000,0.500000,20.830000,0.0,0.060000,0.008000,1.000000,0.000000
25%,20.0,1.539897e+07,139.770000,1.000000,0.500000,2.0,11.000000,0.000000,1.000000,0.500000,60.000000,0.0,10.310000,0.693000,11.000000,48.000000
50%,20.0,2.947232e+07,238.770000,1.000000,0.500000,2.0,14.000000,0.000000,1.000000,0.500000,60.000000,0.0,16.230000,1.002000,14.000000,99.000000
75%,20.0,4.786677e+07,411.770000,2.000000,1.000000,2.0,18.000000,2.743500,2.000000,1.000000,60.000000,0.0,29.620000,1.492000,18.000000,99.000000
max,20.0,6.296289e+07,6700.770000,2.000000,1.000000,2.0,224.000000,71.987000,2.000000,1.000000,70.000000,0.0,40.990000,7.886000,218.000000,99.000000


In [9]:
raw_out_file = 'data/processed/' + raw_filename + '.noNAN.table'
raw_df.to_csv(raw_out_file, sep='\t')

## Append Ground Truth data to the VQSR-filtered table

In [10]:
# Build dictionary of ground truth variants
gt_file = 'data_old/preprocessed/ground_truth_chrom_20.txt'
gt_df = pd.read_csv(gt_file, sep='\t')
gnd_truth_dict = {}

for index, row in gt_df.iterrows(): 
    pos = row['POS']
    ref = row['REF']
    alt = row['ALT']
    gnd_truth_dict[pos] = (ref, alt)

In [11]:
gt_df.shape

(89426, 11)

### Note that there are more ground truth variant calls than variant calls in the non-ground-truth dataset. This reflects the fact that the sequencing pipeline doesn't discover all of the true variants. 

In [12]:
# Initialize the ground truth column to all 0's
df['GROUND_TRUTH'] = 0

# Lookup each variant in the VCF data and label it as a true variant or not
for index, row in df.iterrows():   
    pos = row['POS']
    ref = row['REF']
    alt = row['ALT']
    
    if pos in gnd_truth_dict and gnd_truth_dict[pos] == (ref, alt):
        df.set_value(index, 'GROUND_TRUTH', 1)


In [13]:
filename = filename + '.noNAN.withGndTruthLabels'
out_file_with_labels = 'data/processed/' + filename + '.table'
df.to_csv(out_file_with_labels, sep='\t')

In [14]:
num_true_variants = sum(df['GROUND_TRUTH'])
true_pos_unfiltered = 1.0*num_true_variants/df.shape[0]
print 'Sensitivity (True Pos Rate) Before Any Filtering: ', true_pos_unfiltered

Sensitivity (True Pos Rate) Before Any Filtering:  0.834356252087


### Important! 83.4% of the variants called in the chromosome 20 VCF file are true variants according to the ground truth. So any nontrivial algorithm should have a specificity less than 17%. If we assume that our VCF file contains all of the true pos variants, then our algorithm should attain a sensitivity > 84%.

## Split data into training (70%) and test (30%) sets 

In [16]:
num_samples = df.shape[0]
idx_train_test = train_test_split(range(num_samples), range(num_samples), test_size = 0.3)

In [18]:
idx_train = idx_train_test[0]
idx_test = idx_train_test[1]
assert bool(set(idx_train) & set(idx_test)) == False

df_train = df.loc[idx_train,:]
df_test = df.loc[idx_test,:]

num_train = df_train.shape[0]
num_test = df_test.shape[0]

assert num_train + num_test == num_samples

print 'Training set size: ' + str(num_train)
print 'Test set size: ' + str(num_test)

 Training set size: 60790
Test set size: 26053


## Save the training and test sets

In [20]:
df_train.to_csv('data/processed/' + filename + '.train.table', sep='\t')
df_test.to_csv('data/processed/' + filename + '.text.table', sep='\t')

In [58]:
# Everything below this is old

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [32]:
def num_header_lines(path):
    '''
    Determine the number of header lines in file at `path`
    '''
    N = 0
    with open(path, 'r') as f:
        for line in f:
            if line[0] == '#':
                N += 1
            else:
                break
    return N        

def parse_attributes(attributes):
    for attr in attributes:
        if "=" in attr:
            [key, val] = attr.split('=')
            yield (key, val)

def get_info_field_names(df):
    '''
    VCF records contain an INFO column with several fields (e.g. strand bias)
    separated by semicolons. This function gets the names of all of these fields.
    '''
    row_info = df['INFO'][0].split(';')
    names = set()
    for (key, val) in parse_attributes(row_info):
        names.add(key)
    return names

In [33]:
filename = 'data/VQSRfilter/NA12878.LowSeq.illumina.bwa.sorted.dedup.20.sam.wFlag.qual.raw.snps.vqsr.recal.vcf'
num_header_lines = num_header_lines(filename)
print num_header_lines

124


In [34]:
#filename = 'data/VQSRfilter/NA12878.LowSeq.illumina.bwa.sorted.dedup.20.sam.wFlag.qual.raw.snps.vqsr.recal copy.vcf'

df = pd.read_csv(filename, sep='\t', skiprows=num_header_lines-1)
cols = set(df.columns)
cols -= {'INFO'}

# Get field names in the INFO column, except for non-numeric columns (consider accounting for these later on)
ignore_info_fields = {'DB', 'POSITIVE_TRAIN_SITE', 'NEGATIVE_TRAIN_SITE', 'culprit'}
info_field_names = get_info_field_names(df) - ignore_info_fields

cols |= info_field_names
cols |= {'PASSED_VQSR'}  # binary label indicating if variant passed VQSR filter
cols |= {'GROUND_TRUTH'}  # binary label indicating if the variant is true according to ground truth

## Determine what percentage of called variants are multiallelic.

In [35]:
num_variants = df.shape[0]  # total number of variant calls
num_multi_base_variants = 0
multi_base_variant_rows = []
num_multi_base_ref = 0

for index, row in df.iterrows(): 
    ref = row['REF']
    alt = row['ALT']
    if len(ref) > 1:
        print 'ref: ', ref
        num_multi_base_ref += 1
    if len(alt) > 1:
        print 'alt: ', alt
        num_multi_base_variants += 1
        multi_base_variant_rows.append(index)

print 'Total # of variant calls:', num_variants
print '# of positions (among the variant call sites) where the reference genome has > 1 base:', num_multi_base_ref
print '# of variant calls with > 1 base:', num_multi_base_variants
print '% of variant calls with > 1 base:', num_multi_base_variants/num_variants 

alt:  A,T
alt:  A,G
alt:  G,T
alt:  A,G
alt:  A,G
alt:  A,G
alt:  G,C
alt:  C,T
alt:  A,G
alt:  A,G
alt:  A,G
alt:  C,T
alt:  A,G
alt:  C,G
alt:  G,T
alt:  T,G
alt:  C,G
alt:  A,G
Total # of variant calls: 86825
# of positions (among the variant call sites) where the reference genome has > 1 base: 0
# of variant calls with > 1 base: 18
% of variant calls with > 1 base: 0


### Clearly, there are VERY FEW occurences where a variant is called with more than 1 base. Since these cases are so rare (and probably have different statistics), I'll omit them from the analysis

## Copy the features into a new data frame that only contains the SNP variant calls (no multi base variant calls)

In [37]:
num_rows = df.shape[0]-num_multi_base_variants
df_new = pd.DataFrame(np.zeros(shape=(num_rows, len(cols))), columns=list(cols))

multi_base_variant_rows = set(multi_base_variant_rows)
    
# Copy over all columns except for INFO and FILTER (i.e. the annotations and the VQSR filter result)
for col in df.columns:
    if col in cols and col != 'INFO' and col != 'FILTER':
        df_new[col] = df[col]
    
# Copy over the INFO and FILTER data
for index, row in df.iterrows():   
    if index not in multi_base_variant_rows:
        attrs = row['INFO'].split(';')
        for (key, val) in parse_attributes(attrs):
            if key in cols:
                # Note: this will cause an error if you include the multi base variants. These variants have multiple
                # files for some attributes, so they can't be directly stored into a data frame
                df_new.set_value(index, key, val)

        passed_VQSR = row['FILTER']
        if passed_VQSR == 'PASS':
            df_new.set_value(index, 'PASSED_VQSR', 1)
        else:
            df_new.set_value(index, 'PASSED_VQSR', 0)    

# Obtain Ground Truth Labels

### Note: You must preprocess the ground truth file before running this (see 'preprocess_ground_truth.ipynb'. This ensures that only variants on chromosome 20 are present in the ground truth file.

In [38]:
# Build dictionary of ground truth variants
gt_file = 'data/preprocessed/ground_truth_chrom_20.txt'
gt_df = pd.read_csv(gt_file, sep='\t')
gnd_truth_dict = {}

for index, row in gt_df.iterrows(): 
    pos = row['POS']
    ref = row['REF']
    alt = row['ALT']
    gnd_truth_dict[pos] = (ref, alt)
       

In [39]:
gt_df.shape

(89426, 11)

In [40]:
# Lookup each variant in the VCF data and label it as a true variant or not
for index, row in df_new.iterrows():   
    pos = row['POS']
    ref = row['REF']
    alt = row['ALT']
    
    if pos in gnd_truth_dict and gnd_truth_dict[pos] == (ref, alt):
        df_new.set_value(index, 'GROUND_TRUTH', 1)
    else:
        df_new.set_value(index, 'GROUND_TRUTH', 0)

In [26]:
df_new.to_csv('data/preprocessed/vcf_features_with_labels.txt', sep='\t')

### Display the names of features that may be used for filtering. Include the annotations in the INFO field and the quality score (QUAL). Later, consider incorporating the REF and ALT fields as well.

### Note: VQSLOD (variant quality score log-odds) is the score that VQSR gave to the variant. So this should NOT be used as a feature!

In [29]:
feature_names = info_field_names | {'QUAL'}
feature_names -= {'VQSLOD'}
feature_names

{'AC',
 'AF',
 'AN',
 'BaseQRankSum',
 'ClippingRankSum',
 'DP',
 'FS',
 'MLEAC',
 'MLEAF',
 'MQ',
 'MQ0',
 'MQRankSum',
 'QD',
 'QUAL',
 'ReadPosRankSum',
 'SOR'}

### Summary Statistics

In [42]:
df_new.describe()

,PASSED_VQSR,BaseQRankSum,MQRankSum,VQSLOD,AN,QUAL,FS,#CHROM,DP,ReadPosRankSum,...,MLEAF,MQ0,AF,POS,FILTER,SOR,QD,MQ,GROUND_TRUTH,ClippingRankSum
count,86825.000000,86822.000000,86822.000000,86824.000000,86825.000000,86807.000000,86825.000000,86807.0,86825.000000,86822.000000,...,86825.000000,86825.0,86825.000000,8.680700e+04,86807.0,86825.000000,86825.000000,86825.000000,86825.000000,86822.000000
mean,0.785960,-0.015043,-0.359078,13.971209,1.999585,305.612855,2.124049,20.0,16.942793,0.069032,...,0.658555,0.0,0.658601,3.087120e+07,0.0,1.175032,18.543335,57.862835,0.834495,-0.011704
std,0.410157,1.206852,1.212225,10.259192,0.028794,285.869050,3.937828,0.0,13.388620,0.836458,...,0.232899,0.0,0.232917,1.856200e+07,0.0,0.772055,10.362008,5.171966,0.371638,0.732007
min,0.000000,-6.280000,-8.498000,-81.870000,0.000000,10.200000,0.000000,20.0,0.000000,-4.343000,...,0.000000,0.0,0.000000,6.179500e+04,0.0,0.000000,0.000000,0.000000,0.000000,-3.575000
25%,1.000000,NaN,NaN,NaN,2.000000,NaN,0.000000,NaN,11.000000,NaN,...,0.500000,0.0,0.500000,NaN,NaN,0.693000,10.310000,60.000000,1.000000,NaN
50%,1.000000,NaN,NaN,NaN,2.000000,NaN,0.000000,NaN,14.000000,NaN,...,0.500000,0.0,0.500000,NaN,NaN,1.002000,16.230000,60.000000,1.000000,NaN
75%,1.000000,NaN,NaN,NaN,2.000000,NaN,2.745000,NaN,18.000000,NaN,...,1.000000,0.0,1.000000,NaN,NaN,1.492000,29.620000,60.000000,1.000000,NaN
max,1.000000,6.495000,4.704000,25.200000,2.000000,6700.770000,71.987000,20.0,224.000000,3.963000,...,1.000000,0.0,1.000000,6.294058e+07,0.0,7.886000,40.990000,70.000000,1.000000,3.366000


In [60]:
df_new['BaseQRankSum'].isnull()
print df_new['BaseQRankSum'][86820:]
print df['INFO'][86820]
#df['BaseQRankSum'][86820]


86820      NaN
86821    0.365
86822    1.666
86823      NaN
86824   -2.285
Name: BaseQRankSum, dtype: float64
AC=2;AF=1.00;AN=2;DB;DP=4;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=30.77;MQ0=0;QD=25.51;SOR=1.609;VQSLOD=-2.135e+00;culprit=MQ


In [61]:
# TODO: Account for VQSR tranches!!!

## 83.46% of calls in the VCF file are also in the ground truth file. 78.6% of calls in the VCF file passed the VQSR filter (assuming the highest possible threshold).

## TODO: determine number of variants in ground truth that are not in the VCF file

In [71]:
df_new.shape

(86825, 27)

In [22]:
df_new.shape[0] * 0.7

60777.49999999999

#### There are 86825 total variant calls.  If we use a 70% training set, we can train on about 60,000 samples.

# Split VCF data into training (70%) and test (30%) sets 

In [ ]:
num_samples = df_new.shape[0]
idx_train_test = train_test_split(range(num_samples), range(num_samples), test_size = 0.3)

In [ ]:
idx_train = idx_train_test[0]
idx_test = idx_train_test[1]

df_train = df_new.loc[idx_train,:]
df_test = df_new.loc[idx_test,:]

num_train = df_train.shape[0]
num_test = df_test.shape[0]

# Check that the splitting worked
assert num_train + num_test == num_samples  # correct number of train and test
assert bool(set(idx_train) & set(idx_test)) == False  # train and test indices are different

df_train.to_csv('data/preprocessed/vcf_features_train.txt', sep='\t')
df_test.to_csv('data/preprocessed/vcf_features_test.txt', sep='\t')

In [70]:
df_new.columns

Index([u'PASSED_VQSR', u'AC', u'BaseQRankSum', u'MQRankSum', u'FORMAT',
       u'VQSLOD', u'AN', u'QUAL', u'FS', u'#CHROM', u'REF', u'ID', u'DP',
       u'ReadPosRankSum', u'MLEAC', u'MLEAF', u'MQ0', u'AF', u'POS', u'FILTER',
       u'SOR', u'sample1', u'QD', u'MQ', u'ALT', u'GROUND_TRUTH',
       u'ClippingRankSum'],
      dtype='object')